In [1]:
!nvidia-smi

Tue Dec  8 15:09:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-gpu


     |████████████████████████████████| 320.4MB 54kB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Dataset/train.csv")
store_df = pd.read_csv("/content/drive/MyDrive/Dataset/store.csv")
df = pd.merge(df, store_df, how='inner', on='Store')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


# Data Preprocessing

In [7]:
df = df.drop(df[df.Open == 0].index)
df = df.drop(df[df.Sales == 0].index)

In [8]:
# Convert for time series
df['Date'] = df['Date'].astype('datetime64[ns]')

In [11]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [12]:
df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].median(), inplace = True)

In [13]:
df.fillna(0, inplace = True)

In [ ]:
df['StateHoliday'] = df.StateHoliday.replace([0, '0'], np.nan)
df.StateHoliday.unique()

In [ ]:
dummy_columns = ['StoreType', 'Assortment', 'StateHoliday']
df = pd.get_dummies(df, columns=dummy_columns)

In [ ]:
#Splitting the Date as Day, month and year and adding 3 new columns
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Weekofyear'] = df['Date'].dt.weekofyear

In [ ]:
#How long the competition has been open
df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)

In [ ]:
#How long the promo has been running
df['PromoOpen'] = 12 * (df.Year - df.Promo2SinceYear) + (df.Weekofyear - df.Promo2SinceWeek) / 4.0

In [ ]:
df = df.drop(columns=['Weekofyear','Promo2SinceWeek', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'CompetitionOpenSinceYear', 'Open', 'PromoInterval'])

In [ ]:
#replace competitiondistance with furthest variable
df['CompetitionDistance'] = df.CompetitionDistance.replace(np.nan, df['CompetitionDistance'].max())
                                                           
#Competittion Open - use mean
df['CompetitionOpen'] = df.CompetitionOpen.replace(np.nan, df['CompetitionOpen'].max())

#PromoOpen - recode missing as 0
df['PromoOpen'] = df.PromoOpen.replace(np.nan, 0)

In [26]:
from scipy.stats import zscore

z_scores = np.abs(zscore(df.Sales))
z_scores

array([0.54544483, 0.62373562, 0.70041548, ..., 0.88728236, 0.7654967 ,
       0.6056933 ])

In [28]:
removed_df = df[z_scores < 3]
print('Removed data', len(df) - len(removed_df))
df = removed_df

Removed data 13443


In [30]:
X = df.drop(columns=['Date','Sales','Customers','Store','StateHoliday_b','StateHoliday_c'])
Y = df['Sales']

In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)
scaled_X

array([[-1.72816544,  2.04438985, -0.54018005, ...,  1.74810654,
         1.12330986, -0.00658396],
       [-1.72816544,  2.04438985, -0.54018005, ...,  1.63276315,
         1.12330986, -0.13931366],
       [-1.72816544,  2.04438985, -0.54018005, ...,  1.51741976,
         1.12330986, -0.16263587],
       ...,
       [-0.825448  ,  2.04438985, -0.56192732, ..., -1.36616493,
        -0.89022632, -0.91643385],
       [-0.825448  ,  2.04438985, -0.56192732, ..., -1.48150831,
        -0.89022632, -1.18541212],
       [-0.825448  ,  2.04438985, -0.56192732, ..., -1.5968517 ,
        -0.89022632, -0.89344985]])

In [32]:
from sklearn.model_selection import train_test_split

x_train, x_rest, y_train, y_rest = train_test_split(scaled_X, Y, test_size=0.30)
x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5)
print(len(x_train) + len(x_test) + len(x_val) == len(df))
print(x_train)
print(x_test)
x_val

True
[[-1.2101111  -0.4891435   0.90153635 ... -0.78944799 -0.89022632
  -0.32066056]
 [ 0.6108344   2.04438985  0.97701219 ...  1.05604621 -0.89022632
   0.95195155]
 [ 0.87451475 -0.4891435  -0.56576508 ... -0.90479138 -0.89022632
  -1.28600492]
 ...
 [ 0.97378265  2.04438985 -0.20885392 ... -0.44341783 -0.89022632
   0.37672988]
 [ 0.86520839 -0.4891435   1.16506215 ...  0.59467266  1.12330986
  -0.8937481 ]
 [ 0.61393652 -0.4891435  -0.69624873 ... -0.44341783 -0.89022632
  -1.41309092]]
[[ 1.01411023  2.04438985 -0.64124091 ...  1.63276315 -0.89022632
  -1.48767904]
 [-1.60097845 -0.4891435  -0.66554669 ...  0.71001605  1.12330986
  -0.11908585]
 [-0.96194136  2.04438985 -0.65915043 ...  0.13329911 -0.89022632
  -1.17336497]
 ...
 [ 0.08967793 -0.4891435  -0.66042968 ...  1.28673299  1.12330986
   2.77684763]
 [-0.20192152 -0.4891435  -0.55553106 ...  0.47932927  1.12330986
   0.33617757]
 [ 1.33052666 -0.4891435  -0.48645149 ...  1.05604621 -0.89022632
  -1.04115687]]


array([[ 1.26848422, -0.4891435 , -0.24723146, ..., -1.48150831,
         1.12330986,  0.2539842 ],
       [ 1.56318579, -0.4891435 , -0.6783392 , ..., -1.36616493,
         1.12330986,  1.46883208],
       [ 1.6066155 , -0.4891435 , -0.69113172, ..., -1.36616493,
         1.12330986, -1.09128462],
       ...,
       [ 0.23857978, -0.4891435 ,  2.81913429, ..., -0.32807444,
        -0.89022632, -0.19982556],
       [-0.39735519, -0.4891435 ,  0.94375165, ..., -0.55876121,
        -0.89022632,  1.36327399],
       [-1.38382992, -0.4891435 , -0.49412699, ...,  1.51741976,
        -0.89022632, -0.71756549]])

In [33]:
x_train.shape

(581626, 9)

In [34]:
x_test.shape

(124634, 9)

## Since the input for a CNN and LSTM should be in 3 dimension, the inputs were reshaped.

In [36]:
# reshape input to be [samples, time steps, features] which is required for LSTM
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])

In [37]:
x_train.shape

(581626, 9, 1)

In [38]:
x_test.shape

(124634, 9, 1)

# Modelling

In [39]:
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import LSTM
from keras.utils import np_utils

In [41]:
model = Sequential()
model.add(LSTM(16,return_sequences=True,input_shape=(9, 1), activation = 'tanh', kernel_initializer = 'glorot_uniform',
              kernel_regularizer = 'l2'))
model.add(Dropout(0.2))
model.add(LSTM(8,return_sequences=True, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(LSTM(8, activation = 'linear'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [42]:
model.compile(loss='mean_squared_error',optimizer='adam', metrics = ['mse'])

In [43]:
lstm_model = model.fit(x_train, y_train, epochs=100, verbose=1)

Epoch 1/100
18176/18176 [==============================] - 196s 11ms/step - loss: 8862101.0000 - mse: 8862101.0000
Epoch 2/100
18176/18176 [==============================] - 204s 11ms/step - loss: 8385684.5000 - mse: 8385684.5000
Epoch 3/100
18176/18176 [==============================] - 195s 11ms/step - loss: 7603792.0000 - mse: 7603792.0000
Epoch 4/100
18176/18176 [==============================] - 194s 11ms/step - loss: 7249530.5000 - mse: 7249530.5000
Epoch 5/100
18176/18176 [==============================] - 195s 11ms/step - loss: 7183652.0000 - mse: 7183652.0000
Epoch 6/100
18176/18176 [==============================] - 193s 11ms/step - loss: 7152332.0000 - mse: 7152330.0000
Epoch 7/100
18176/18176 [==============================] - 205s 11ms/step - loss: 7102688.5000 - mse: 7102688.0000
Epoch 8/100
18176/18176 [==============================] - 203s 11ms/step - loss: 7059755.5000 - mse: 7059754.5000
Epoch 9/100
18176/18176 [==============================] - 197s 11ms/step - loss

In [44]:
predict = model.predict(x_test)

In [49]:
from sklearn import metrics

In [55]:
# Print the results of MAE
print(np.sqrt(metrics.mean_absolute_error(y_test, predict)))

# Print the results of MSE
print(metrics.mean_squared_error(y_test, predict))

# Print the results of RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, predict)))

39.66995854903035
4278108.478787572
2068.3588853938218
